Minikorpuse tegemine DataFrame-iks

In [1]:
import json
import pandas as pd
from collections import defaultdict
import os

In [2]:
def ava_json(failinimi):
    with open(failinimi, "r", encoding = "UTF-8") as f:
        return json.load(f)

In [3]:
df = pd.read_csv("../minikorpus/skoorid.csv")

In [4]:
df_drop = df.drop(["SKOORID", "VASTAJAID", "STD"], axis = 1)

In [5]:
df_form = df_drop[df["DIMENSIOON"] == "form"].drop("DIMENSIOON", axis = 1).fillna(0)
df_spont = df_drop[df["DIMENSIOON"] == "spont"].drop("DIMENSIOON", axis = 1).fillna(0)

In [6]:
sonastik = defaultdict(lambda: dict())

for file in df_form["FAIL"]:
    sonastik[file] = ava_json(os.path.join("../minikorpus/mini_andmed/", file.split(".txt")[0] + "_andmed.json"))
    sonastik[file]["formaalsus"] = list(df_form[df_form["FAIL"] == file]["KESKMINE SKOOR"].items())[0][1]/3
    sonastik[file]["spontaansus"] = list(df_spont[df_spont["FAIL"] == file]["KESKMINE SKOOR"].items())[0][1]/3

In [7]:
df_uus = pd.DataFrame.from_dict(sonastik, orient="index")

In [8]:
df_uus = df_uus.replace(-1, 0)

Lisandi tegemine DataFrame-iks

In [10]:
df = pd.read_csv("../minikorpus/lisa_nimed.txt")

In [11]:
sonastik = defaultdict(lambda: dict())

for _, row in df.iterrows():
    file = row["failinimi"]
    sonastik[file] = ava_json(os.path.join("../minikorpus/lisa_andmed/",  file.split(".xml")[0] + "_andmed.json"))
    sonastik[file]["formaalsus"] = row["formaalsus"]
    sonastik[file]["spontaansus"] = row["spontaansus"]

In [12]:
df_uus2 = pd.DataFrame.from_dict(sonastik, orient="index")

In [13]:
df_uus2 = df_uus2.replace(-1, 0)

DataFrame'id kokku

In [15]:
df_koos = pd.concat([df_uus, df_uus2])

In [16]:
df_koos.to_csv("../minikorpus/Randmestik.csv")